In [15]:
d_data = {
    'class_1': [
        's3://url_s3/unique_id_3.jpg',
        's3://url_s3/unique_id_5.jpg',
    ],
    'class_2': [
        's3://url_s3/unique_id_1.jpg',
        's3://url_s3/unique_id_2.jpg',
        's3://url_s3/unique_id_4.jpg',
        's3://url_s3/unique_id_6.jpg'
    ]
}

list_save_local = [
    'workspace/class_1/unique_id_1.jpg',
    'workspace/class_1/unique_id_2.jpg',
    'workspace/class_1/unique_id_3.jpg',
    'workspace/class_2/unique_id_4.jpg',
    'workspace/class_2/unique_id_5.jpg',
    'workspace/class_2/unique_id_6.jpg'
]


In [16]:
import re
import os

mapped = {}
def get_properties(local_path):
    class_name = local_path.split('/')[-2]
    filename = os.path.basename(local_path)
    ext = filename.split('.')[-1]
    unique_id = filename.split(f'.{ext}')[0]
    ls_urls = d_data[class_name]
    
    url_fix = None
    for urls in ls_urls:
        if unique_id in urls:
            url_fix = urls
            print('find', unique_id, urls)
            break
    return class_name,unique_id,url_fix

for i, local_path in enumerate(list_save_local):
    class_name, unique_id, url_fix = get_properties(local_path)
    print(url_fix)
    mapped[unique_id] = {
        'url': url_fix,
        'class': class_name,
        'local_path': local_path
    }
    break
mapped

None


{'unique_id_1': {'url': None,
  'class': 'class_1',
  'local_path': 'workspace/class_1/unique_id_1.jpg'}}

In [22]:
import os

root_directory = "../../current_dataset_test/"



# for subdir, dirs, files in os.walk(root_directory):
#     for file in files:
#         full_path = os.path.join(subdir, file)
#         print(full_path)

lsfp = [os.path.join(subdir, file) for subdir, _, files in os.walk(root_directory) for file in files]

In [ ]:
lsfp

In [34]:
list_of_softmax = [    [0.2, 0.1, 0.5, 0.1, 0.1],
    [0.3, 0.3, 0.1, 0.1, 0.2],
    [0.1, 0.2, 0.1, 0.4, 0.2],
    [0.2, 0.3, 0.1, 0.2, 0.2],
    [0.4, 0.1, 0.1, 0.1, 0.3],
    [0.2, 0.2, 0.4, 0.1, 0.1],
    [0.1, 0.1, 0.6, 0.1, 0.1],
    [0.3, 0.2, 0.2, 0.2, 0.1],
    [0.1, 0.3, 0.2, 0.2, 0.2],
    [0.2, 0.1, 0.2, 0.2, 0.3],
    [0.1, 0.1, 0.1, 0.5, 0.2],
    [0.2, 0.1, 0.1, 0.2, 0.4],
    [0.2, 0.2, 0.2, 0.2, 0.2],
    [0.4, 0.1, 0.1, 0.1, 0.3],
    [0.2, 0.2, 0.1, 0.2, 0.3],
    [0.1, 0.2, 0.1, 0.3, 0.3],
    [0.2, 0.3, 0.1, 0.1, 0.3],
    [0.3, 0.1, 0.1, 0.4, 0.1],
    [0.2, 0.2, 0.2, 0.1, 0.3],
    [0.1, 0.2, 0.2, 0.2, 0.3]
]

list_ground_truth_label = [2, 0, 3, 1, 4, 2, 2, 0, 1, 4, 3, 4, 2, 4, 1, 3, 1, 3, 4, 4]

In [43]:
len(list_ground_truth_label), len(list_of_softmax)

(20, 20)

In [44]:
import plotly.graph_objs as go
import numpy as np
from sklearn.metrics import roc_curve, auc

# Define the number of classes
num_classes = len(list_of_softmax[0])

# Compute the false positive rate, true positive rate and area under the ROC curve for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve((np.array(list_ground_truth_label) == i).astype(int), np.array(list_of_softmax)[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute the micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve((np.array(list_ground_truth_label) < num_classes).astype(int), np.array(list_of_softmax).ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Create the ROC curve traces for each class and micro-average
roc_curve_traces = []
for i in range(num_classes):
    roc_curve_traces.append(go.Scatter(x=fpr[i], y=tpr[i], name='Class {}'.format(i), line=dict(width=2)))
roc_curve_traces.append(go.Scatter(x=fpr["micro"], y=tpr["micro"], name='Micro-average', line=dict(width=2, dash='dot')))
roc_curve_traces.append(go.Scatter(x=[0, 1], y=[0, 1], name='Reference Line', line=dict(width=2, dash='dash')))

# Create the layout for the ROC curve plot
roc_curve_layout = go.Layout(
    title='ROC Curve',
    xaxis=dict(title='False Positive Rate'),
    yaxis=dict(title='True Positive Rate'),
    width=800,
    height=500
)

# Plot the ROC curve
fig = go.Figure(data=roc_curve_traces, layout=roc_curve_layout)
fig.show()


ValueError: Found input variables with inconsistent numbers of samples: [20, 100]

In [39]:
i = 1
(np.array(list_ground_truth_label) == i).astype(int)

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0])

# Performance

In [50]:
onnx_performance = {'accuracy': 88.7, 'speed': 0.003338, 'vram': 98.0, 'ram': 13.4296875, 'device': 0}
torchscript_performance = {'accuracy': 88.7, 'speed': 0.012237, 'vram': 36.0, 'ram': 71.87109375, 'device': 0}

In [53]:
import plotly.graph_objs as go

# Define the data
onnx_accuracy = onnx_performance['accuracy']
onnx_speed = onnx_performance['speed']
onnx_vram = onnx_performance['vram']
onnx_ram = onnx_performance['ram']

torchscript_accuracy = torchscript_performance['accuracy']
torchscript_speed = torchscript_performance['speed']
torchscript_vram = torchscript_performance['vram']
torchscript_ram = torchscript_performance['ram']

# Convert speed values to percentage of reference
torchscript_speed_percent = 100.0
onnx_speed_percent = (onnx_speed / torchscript_speed) * 100.0

# Define the data traces
data = [
    {
        'x': ['Accuracy', 'Speed', 'VRAM', 'RAM'],
        'y': [onnx_accuracy, onnx_speed_percent, onnx_vram, onnx_ram],
        'name': 'ONNX',
        'type': 'bar',
        'marker': {
            'color': 'rgb(166,206,227)',
            'line': {
                'color': 'rgb(54, 55, 56)',
                'width': 1.5
            }
        },
        'text': ['{:.1f}%'.format(onnx_accuracy), '{:.3f}'.format(onnx_speed), '{:.1f}'.format(onnx_vram), '{:.3f}'.format(onnx_ram)]
    },
    {
        'x': ['Accuracy', 'Speed', 'VRAM', 'RAM'],
        'y': [torchscript_accuracy, torchscript_speed_percent, torchscript_vram, torchscript_ram],
        'name': 'TorchScript',
        'type': 'bar',
        'marker': {
            'color': 'rgb(253,191,111)',
            'line': {
                'color': 'rgb(54, 55, 56)',
                'width': 1.5
            }
        },
        'text': ['{:.1f}%'.format(torchscript_accuracy), '100.0%', '{:.1f}'.format(torchscript_vram), '{:.3f}'.format(torchscript_ram)]
    }
]

# Create the layout for the bar chart
layout = go.Layout(
    title='Comparison of ONNX and TorchScript',
    yaxis=dict(title='Value'),
    width=800,
    height=500,
    plot_bgcolor='rgba(240, 240, 240, 0.95)'
)

# Create the bar chart
fig = go.Figure(data=data, layout=layout)

# Update the font size of the text on the chart
fig.update_layout(
    font=dict(
        size=12
    )
)

# Add a reference line for accuracy
fig.add_shape(
    type='line',
    x0=-0.5,
    y0=torchscript_accuracy,
    x1=3.5,
    y1=torchscript_accuracy,
    line=dict(
        color='rgb(34, 67, 115)',
        width=2,
        dash='dash'
    )
)

fig.show()


In [58]:
torchscript_speed, onnx_speed, round( onnx_speed/ torchscript_speed * 100.0, 2)

(0.012237, 0.003338, 27.28)

In [77]:
compare(100,89)

89.0


89.0

In [92]:
import sys
sys.path.append('/home/agfian/shared/hdd_2/common-project/classifier/pytorch-lighting-image-classifier')

torchscript_performance = {'accuracy': 84.4, 'speed': 0.011897, 'total_prediction': 35.714, 'vram': 36.0, 'ram': 7.9609375, 'device': 0}
onnx_performance = {'accuracy': 84.4, 'speed': 0.003365, 'total_prediction': 10.107, 'vram': 98.0, 'ram': 6.6484375, 'device': 0}

from src.utils import make_graph_performance

fig = make_graph_performance(
    torchscript_performance=torchscript_performance, 
    onnx_performance=onnx_performance)
fig.show()


Generating Graph Performance


In [78]:
import torch

In [82]:
preds_top1 = torch.tensor([13,  9,  6,  6,  6,  6])
preds_epoch = torch.tensor([
        [ 0.2583, -0.2094,  0.0035,  0.2264,  0.2266,  0.0683,  0.4834, -0.0780,
          0.0547,  0.2859,  0.0514,  0.2844, -0.2450,  0.6323,  0.4397],
        [-0.3323,  0.1292,  0.1829, -0.0155,  0.1655, -0.3525, -0.1748, -0.0070,
         -0.3491,  0.5298,  0.4387, -0.0216,  0.0522, -0.2408, -0.2362],
        [ 0.1615, -0.2820,  0.0427,  0.1918,  0.0870, -0.0009,  0.4656, -0.0912,
          0.1660,  0.1104,  0.0715,  0.2220, -0.2859,  0.4480,  0.2944],
        [ 0.1981, -0.2998,  0.0202,  0.2285,  0.1458,  0.1176,  0.5938, -0.0811,
          0.1278,  0.0952,  0.0176,  0.2281, -0.2800,  0.4812,  0.3176],
        [ 0.1726, -0.3484,  0.0190,  0.2727,  0.1177,  0.0470,  0.5688,  0.0581,
          0.1323, -0.0130,  0.0924,  0.2251, -0.3369,  0.5469,  0.2581],
        [ 0.2081, -0.2620, -0.0505,  0.2249,  0.1096,  0.1001,  0.5684, -0.0972,
          0.1036,  0.1022,  0.0123,  0.2417, -0.2859,  0.4727,  0.2666]],
        dtype=torch.float16)